Params to change:

In [ ]:
non_freezing_layers = ["layer.4", "layer.5", "layer.6", "layer.7", "layer.8"]

mlqa_eval_languages = ["mlqa.en.en", "mlqa.ar.en", "mlqa.en.ar", "mlqa.ar.ar", "mlqa.es.es", "mlqa.en.de", "mlqa.de.de"]

batch_size = 16

model_name = 'xlm-roberta-base'
#model_name = 'salti/bert-base-multilingual-cased-finetuned-squad' # This model is already a finetuned BERT for QA

Install dev-version of Hugginface transformers from GitHub-repo, as well as Hugginface datasets:

In [ ]:
!pip install datasets
!git clone https://github.com/huggingface/transformers
!cd transformers && pip install .

     |████████████████████████████████| 225kB 13.2MB/s 
     |████████████████████████████████| 112kB 24.4MB/s 
     |████████████████████████████████| 245kB 25.3MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 72664, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (376/376), done.
remote: Total 72664 (delta 392), reused 489 (delta 286), pack-reused 71962
Receiving objects: 100% (72664/72664), 56.22 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (51520/51520), done.
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 14.6MB/s 
     |████████████████████████████████| 3.3MB 27.0MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2259809 sha256=2f39c7123ce83b239f5c8d1c8830ada78ae62447202023d8f23e7d13b2e573ee
  Stored in directory: /tmp/pi

# Run Experiment

This code is based of [here](https://github.com/huggingface/transformers/tree/master/examples/pytorch/question-answering) -- the Hugginface QA-environment. We pull a modified version of `run_qa.py` from our GitHub-repo, but use the standard `trainer.py` and `utils_qa.py`.



In [1]:
!curl -L -O https://raw.githubusercontent.com/jakobwes/QA_layer_freeze/main/experiment_code/run_qa.py
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/question-answering/trainer_qa.py
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/question-answering/utils_qa.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28626  100 28626    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4007  100  4007    0     0  26536      0 --:--:-- --:--:-- --:--:-- 26536
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22276  100 22276    0     0   150k      0 --:--:-- --:--:-- --:--:--  148k


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
import os

# Model path and folder name (without slash)
model_saving_folder_name = "./"+model_name
model_saving_path = model_saving_folder_name+"/"

# Get and save model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model.save_pretrained(model_saving_path)

# Get and save tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)  
tokenizer.save_pretrained(model_saving_path)

# Output dir path and output dir name (without slash) for squad-trained model
model_output_dir_name = model_saving_folder_name+"_ft"
model_output_dir_path = model_saving_folder_name+"_ft/"

batch_size_str = str(batch_size)



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

We need to freeze a majority of the layers. For that we need to replace the lines 547-548 in `run_qa.py` by the following snippet:

In [ ]:
for name, param in model.base_model.named_parameters():
  if not any(s in name for s in non_freezing_layers):
    param.requires_grad = False

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

Run training and evaluation:

In [ ]:
# Run training and save in output-dir
!python run_qa.py  \
    --model_name_or_path $model_saving_folder_name \
    --output_dir $model_output_dir_name \
    --num_train_epochs 3.0 \
    --dataset_name squad \
    --do_train \
    --do_eval \
    --per_device_train_batch_size $batch_size_str \
    --per_device_eval_batch_size $batch_size_str \
    --max_seq_length 384 \
    --doc_stride 128

for language_pair in mlqa_eval_languages:

  escaped_language_pair_str = language_pair.replace(".", "_")
  eval_output_dir = "./eval/" + model_name + "/" + escaped_language_pair_str + "/"

  # Run evaluation with weights from output_dir_model and save in eval_output_dir
  !python run_qa.py  \
      --model_name_or_path $model_output_dir_path \
      --output_dir $eval_output_dir \
      --dataset_name mlqa \
      --dataset_config_name $language_pair \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size $batch_size_str \
      --per_device_eval_batch_size $batch_size_str \
      --max_seq_length 384 \
      --doc_stride 128

2021-05-15 14:25:47.144414: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/15/2021 14:25:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/15/2021 14:25:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./mBERT_ft, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May15_14-25-49_07ef643d1386, logging_strategy=IntervalStrategy.STEPS, logging_first_step=Fals

# Retrieve Results

Extract the result and save them in a dict:

In [ ]:
import json

results = {}
results["mlqa"] = {}
results["squad"] = {}

# Extract mlqa-results
results_mlqa = {}
for language_pair in mlqa_eval_languages:

  results_per_language_pair = {}

  escaped_language_pair_str = language_pair.replace(".", "_")
  eval_output_dir = "./eval/" + model_name + "/" + escaped_language_pair_str

  with open(eval_output_dir+"/all_results.json") as json_file:
    results_per_language_pair["all results"] = json.load(json_file)

  with open(eval_output_dir+"/eval_results.json") as json_file:
    results_per_language_pair["eval results"] = json.load(json_file)

  with open(eval_output_dir+"/predict_results.json") as json_file:
    results_per_language_pair["predict results"] = json.load(json_file)
    
  results_mlqa[escaped_language_pair_str] = results_per_language_pair

results["mlqa"] = results_mlqa


# Extract squad results
results_squad = {}


model_saving_folder_name = "./"+model_name
model_output_dir_path = model_saving_folder_name+"_ft/"

with open(model_output_dir_path+"/all_results.json") as json_file:
  results_squad["all results"] = json.load(json_file)

with open(model_output_dir_path+"/eval_results.json") as json_file:
  results_squad["eval results"] = json.load(json_file)

with open(model_output_dir_path+"/train_results.json") as json_file:
  results_squad["train results"] = json.load(json_file)
  
results["squad"] = results_squad


In [ ]:
results